In [105]:
### GROUP 2 ###

# B.J.T. Donkers - 1506897
# S. van der Leek - 1564226
# L. Sloot - 1365711
# S. Diemel  - 1645749
# A.E. Murariu - 1532081
# Z.V. Petrov - 1554999

In [106]:
# imports
import pandas as pd

In [107]:
# read the data (2012)
df = pd.read_csv('data/BPI_Challenge_2012-training.csv')

df_test = pd.read_csv('data/BPI_Challenge_2012-test.csv')

amount_column_in_data = True
# read the data (road traffic fines)
# df = pd.read_csv('data/Road_Traffic_Fine_Management_Process-training.csv')

# read the data (2017)
# df = pd.read_csv('data/BPI Challenge 2017-training.csv')

# read the data (2018)
# df = pd.read_csv('data/BPI Challenge 2018-training.csv')

df

,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp
0,0,173688,2011-10-01T00:38:44.546+02:00,20000,A_SUBMITTED,COMPLETE,01-10-2011 00:38:44.546
1,1,173688,2011-10-01T00:38:44.546+02:00,20000,A_PARTLYSUBMITTED,COMPLETE,01-10-2011 00:38:44.880
2,2,173688,2011-10-01T00:38:44.546+02:00,20000,A_PREACCEPTED,COMPLETE,01-10-2011 00:39:37.906
3,3,173688,2011-10-01T00:38:44.546+02:00,20000,W_Completeren aanvraag,SCHEDULE,01-10-2011 00:39:38.875
4,4294967296,173691,2011-10-01T08:08:58.256+02:00,5000,A_SUBMITTED,COMPLETE,01-10-2011 08:08:58.256
...,...,...,...,...,...,...,...
214372,38835094290529,201854,2012-01-18T02:09:07.029+01:00,50000,O_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214373,38835094290528,201854,2012-01-18T02:09:07.029+01:00,50000,A_CANCELLED,COMPLETE,14-03-2012 15:30:19.361
214374,38835094290530,201854,2012-01-18T02:09:07.029+01:00,50000,W_Nabellen incomplete dossiers,COMPLETE,14-03-2012 15:30:23.187
214375,35858681954366,199678,2012-01-10T19:16:52.800+01:00,30000,W_Nabellen offertes,START,14-03-2012 15:36:15.299


In [108]:
# Defining database-specific variables
case_column = "case concept:name"
registration_time_column = "case REG_DATE"
event_column = "event concept:name"
timestamp_column = "event time:timestamp"
amount_column = 'case AMOUNT_REQ'
row_nr_column = 'row_nr'
index_column = "index"
timeformat_registration = "%Y-%m-%dT%H:%M:%S" # new time format
timeformat_timestamp = "%d-%m-%Y %H:%M:%S.%f"
additionalInfo = 'event lifecycle:transition' # additional info for the timePrediction_onTheFly, set to None if you don't want to use it

# Names of new columns we will add in the notebooks
position_column = "Position" # added in the baseline notebook
baseline_next_event_column = "Baseline Prediction for Next Activity" # added in the baseline notebook
baseline_next_timestamp_column = "Baseline Prediction for Next Timestamp" # added in the baseline notebook
pred_event_otf = "pred_event_otf" # added by event_onthefly notebook
otf_next_timestamp_column = "OnTheFly Prediction for Next Timestamp" # added by the timePrediction_onTheFly notebook

In [109]:
# Running the notebook that adds the baseline predictions to df and df_test
%run -i "Baseline Prediction Tool.ipynb"

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 214377 entries, 0 to 214376
Data columns (total 8 columns):
 #   Column                      Non-Null Count   Dtype         
---  ------                      --------------   -----         
 0   index                       214377 non-null  int64         
 1   eventID                     214377 non-null  int64         
 2   case concept:name           214377 non-null  int64         
 3   case REG_DATE               214377 non-null  datetime64[ns]
 4   case AMOUNT_REQ             214377 non-null  int64         
 5   event concept:name          214377 non-null  object        
 6   event lifecycle:transition  214377 non-null  object        
 7   event time:timestamp        214377 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(4), object(2)
memory usage: 13.1+ MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 47823 entries, 0 to 47822
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype        

In [110]:
# Accuracy % of the baseline event predictions:

training_event_accuracy = len(df[(df[baseline_next_event_column].shift(periods=1)==df[event_column]) & (df[case_column].shift(periods=1) == df[case_column])]) * 100 / len(df[df[case_column].shift(periods=1) == df[case_column]])

test_event_accuracy = len(df_test[(df_test[baseline_next_event_column].shift(periods=1)==df_test[event_column]) & (df_test[case_column].shift(periods=1) == df_test[case_column])]) * 100 / len(df_test[df_test[case_column].shift(periods=1) == df_test[case_column]])

training_event_accuracy, test_event_accuracy

(64.9130980638327, 64.06370976661874)

In [111]:
# Mean Absolute Error of the baseline time predictions:

training_time_MAE = abs(df[timestamp_column] - df[baseline_next_timestamp_column].shift(periods=1))[df[case_column].shift(periods=1) == df[case_column]].mean()

test_time_MAE = abs(df_test[timestamp_column] - df_test[baseline_next_timestamp_column].shift(periods=1))[df_test[case_column].shift(periods=1) == df_test[case_column]].mean()

training_time_MAE, test_time_MAE

(Timedelta('0 days 16:20:59.739888261'),
 Timedelta('0 days 14:43:13.416911984'))

In [112]:
# Running the notebook that adds the on-the-fly event prediction to df and df_test
df = df.sort_values([index_column])
df.reset_index(inplace=True, drop=True)
df_test = df_test.sort_values([index_column])
df_test.reset_index(inplace=True, drop=True)
%run -i "event_onTheFly.ipynb"

C:\Users\20203666\AppData\Local\Temp/ipykernel_16208/3488171749.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[row_nr_column] = dataset.index
C:\Users\20203666\AppData\Local\Temp/ipykernel_16208/3488171749.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_test[row_nr_column] = dataset_test.index
C:\Users\20203666\AppData\Local\Temp/ipykernel_16208/3488171749.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

   case concept:name      event concept:name event lifecycle:transition  \
0             173688             A_SUBMITTED                   COMPLETE   
1             173688       A_PARTLYSUBMITTED                   COMPLETE   
2             173688           A_PREACCEPTED                   COMPLETE   
3             173688  W_Completeren aanvraag                   SCHEDULE   
4             173691             A_SUBMITTED                   COMPLETE   

   case AMOUNT_REQ  row_nr  
0            20000       0  
1            20000       1  
2            20000       2  
3            20000       3  
4                0       4  
dict: Elapsed time during the whole program in seconds: 13.265625
train accuracy:0.8166329913490398
length df = 214377, length predictions = 214377
test accuracy:0.8156398628470302
length df = 47823, length predictions = 47823


In [120]:
print(f'train accuracy event on the fly: {train_acc_event_otf*100}') # does still run because is in event_onthefly.ipynb
print(f'test accuracy event on the fly: {test_acc_event_otf*100}')

train accuracy event on the fly: 81.66329913490398
test accuracy event on the fly: 81.56398628470302


In [114]:
#df = df.sort_values(by=[case_column, timestamp_column]).reset_index(drop=True)
#df_test = df_test.sort_values(by=[case_column, timestamp_column]).reset_index(drop=True)

#training_event_accuracy = len(df[(df[pred_event_otf].shift(periods=1)==df[event_column]) & (df[case_column].shift(periods=1) == df[case_column])]) * 100 / len(df[df[case_column].shift(periods=1) == df[case_column]])

#test_event_accuracy = len(df_test[(df_test[pred_event_otf].shift(periods=1)==df_test[event_column]) & (df_test[case_column].shift(periods=1) == df_test[case_column])]) * 100 / len(df_test[df_test[case_column].shift(periods=1) == df_test[case_column]])

#training_event_accuracy, test_event_accuracy


In [115]:
# Accuracy % of the on-the-fly event predictions:

# training_event_accuracy = len(df[(df[baseline_next_event_column].shift(periods=1)==df[event_column]) & (df[case_column].shift(periods=1) == df[case_column])]) * 100 / len(df[df[case_column].shift(periods=1) == df[case_column]])

# test_event_accuracy = len(df_test[(df_test[baseline_next_event_column].shift(periods=1)==df_test[event_column]) & (df_test[case_column].shift(periods=1) == df_test[case_column])]) * 100 / len(df_test[df_test[case_column].shift(periods=1) == df_test[case_column]])

# training_event_accuracy, test_event_accuracy

In [116]:
# Running the notebook that adds the on-the-fly event prediction to df and df_test
%run -i "timePrediction_onTheFly.ipynb" # Takes around 2 minutes to run

   case concept:name      event concept:name    event time:timestamp  \
0             173688             A_SUBMITTED 2011-10-01 00:38:44.546   
1             173688       A_PARTLYSUBMITTED 2011-10-01 00:38:44.880   
2             173688           A_PREACCEPTED 2011-10-01 00:39:37.906   
3             173688  W_Completeren aanvraag 2011-10-01 00:39:38.875   
4             173691             A_SUBMITTED 2011-10-01 08:08:58.256   

  event lifecycle:transition  row_nr  
0                   COMPLETE       0  
1                   COMPLETE       1  
2                   COMPLETE       2  
3                   SCHEDULE       3  
4                   COMPLETE       4  


C:\Users\20203666\AppData\Local\Temp/ipykernel_16208/1376477422.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset[row_nr_column] = dataset.index
C:\Users\20203666\AppData\Local\Temp/ipykernel_16208/1376477422.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dataset_test[row_nr_column] = dataset_test.index
C:\ProgramData\Anaconda3\envs\my_env\lib\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .lo

dict: Elapsed time during the whole program in seconds: 55.6875


C:\Users\20203666\AppData\Local\Temp/ipykernel_16208/2674699106.py:14: RuntimeWarning: divide by zero encountered in log2
  error_df['log_predictionError'] = [np.log2(x) for x in error_df['predictionError']]


The mean absolute error on the training-set is: 725.8 minutes.
dict: Elapsed time during the whole program on the testset in seconds: 3.734375
The mean absolute error on the test-set is: 619.86 minutes.
                         event_column               timestamp  \
0        A_SUBMITTED_XX_COMPLETE_XX_0 2012-02-03 17:17:11.047   
1  A_PARTLYSUBMITTED_XX_COMPLETE_XX_0 2012-02-03 17:17:11.323   
3        A_SUBMITTED_XX_COMPLETE_XX_0 2012-02-03 17:23:41.949   
4  A_PARTLYSUBMITTED_XX_COMPLETE_XX_0 2012-02-03 17:23:42.504   
5      A_PREACCEPTED_XX_COMPLETE_XX_0 2012-02-03 17:24:23.379   

   predictionError  predictionErrorRatio  predictedTimePassed  timePassed  \
0         0.004439              0.964925             0.009039    0.004600   
1         0.008243              0.015630             0.535593    0.527350   
3         0.000655              0.070834             0.008595    0.009250   
4         0.146482              0.215019             0.534768    0.681250   
5         0.002926   

In [117]:
df.head(10)

,index,eventID,case concept:name,case REG_DATE,case AMOUNT_REQ,event concept:name,event lifecycle:transition,event time:timestamp,Position,Baseline Prediction for Next Activity,Baseline Prediction for Next Timestamp,pred_event_otf
0,0,0,173688,2011-10-01 00:38:44,20000,A_SUBMITTED,COMPLETE,2011-10-01 00:38:44.546,1,A_PARTLYSUBMITTED,2011-10-01 00:38:45.128640462,W_Nabellen offertes
1,1,1,173688,2011-10-01 00:38:44,20000,A_PARTLYSUBMITTED,COMPLETE,2011-10-01 00:38:44.880,2,W_Afhandelen leads,2011-10-01 00:39:19.982779539,W_Nabellen offertes
2,2,2,173688,2011-10-01 00:38:44,20000,A_PREACCEPTED,COMPLETE,2011-10-01 00:39:37.906,3,W_Completeren aanvraag,2011-10-01 00:39:38.408771753,W_Nabellen offertes
3,3,3,173688,2011-10-01 00:38:44,20000,W_Completeren aanvraag,SCHEDULE,2011-10-01 00:39:38.875,4,W_Completeren aanvraag,2011-10-01 17:58:08.032379243,W_Nabellen offertes
4,4,4294967296,173691,2011-10-01 08:08:58,5000,A_SUBMITTED,COMPLETE,2011-10-01 08:08:58.256,1,A_PARTLYSUBMITTED,2011-10-01 08:08:58.838640462,W_Nabellen offertes
5,5,4294967297,173691,2011-10-01 08:08:58,5000,A_PARTLYSUBMITTED,COMPLETE,2011-10-01 08:09:02.195,2,W_Afhandelen leads,2011-10-01 08:09:37.297779539,W_Nabellen offertes
6,6,4294967298,173691,2011-10-01 08:08:58,5000,A_PREACCEPTED,COMPLETE,2011-10-01 08:09:56.648,3,W_Completeren aanvraag,2011-10-01 08:09:57.150771753,W_Nabellen offertes
7,7,4294967299,173691,2011-10-01 08:08:58,5000,W_Completeren aanvraag,SCHEDULE,2011-10-01 08:09:59.578,4,W_Completeren aanvraag,2011-10-02 01:28:28.735379243,W_Nabellen offertes
8,8,8589934592,173694,2011-10-01 08:10:30,7000,A_SUBMITTED,COMPLETE,2011-10-01 08:10:30.287,1,A_PARTLYSUBMITTED,2011-10-01 08:10:30.869640462,W_Nabellen offertes
9,9,8589934593,173694,2011-10-01 08:10:30,7000,A_PARTLYSUBMITTED,COMPLETE,2011-10-01 08:10:30.591,2,W_Afhandelen leads,2011-10-01 08:11:05.693779539,W_Nabellen offertes


In [13]:
# Mean Absolute Error of the on-the-fly time predictions:

# training_time_MAE = abs(df[timestamp_column] - df[baseline_next_timestamp_column].shift(periods=1))[df[case_column].shift(periods=1) == df[case_column]].mean()

# test_time_MAE = abs(df_test[timestamp_column] - df_test[baseline_next_timestamp_column].shift(periods=1))[df_test[case_column].shift(periods=1) == df_test[case_column]].mean()

# training_time_MAE, test_time_MAE